In [1]:
###getting the reports local url's:###
##extracting folders id from shared memory

from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd

# Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"

def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    """Recursively list all files in a folder and subfolders"""
    all_files = []

    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recurse into subfolder
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            else:
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id'],
                    "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Run the recursive function
files_list = list_all_files_in_folder_recursive(folder_id)

# Convert to DataFrame
df_files = pd.DataFrame(files_list)
##df_files.head()


##getting local id of the file
!pip install python-docx
import os
from googleapiclient.http import MediaIoBaseDownload
import io

def get_chapter_file(chapter_num: int, year: str, download_dir="/content") -> str:
    """
    Download a chapter file for a given year from Google Drive and return its local path.

    Args:
        chapter_num (int): The chapter number (e.g., 1, 2, ..., 14).
        year (str): The year folder name.
        download_dir (str): Local folder to save the file. Defaults to /content in Colab.

    Returns:
        str: Local path to the downloaded file, or None if not found.
    """
    # Normalize chapter filename (01, 02, ...)
    chapter_str = f"{chapter_num:02}"

    # Find matching file in df_files
    match = df_files[
        (df_files["file_name"].str.contains(chapter_str)) &
        (df_files["file_path"].str.contains(f"{year}/"))
    ]

    if match.empty:
        print(f"❌ No file found for chapter {chapter_str} in year {year}")
        return None

    file_id = match.iloc[0]["file_id"]
    file_name = match.iloc[0]["file_name"]
    local_path = os.path.join(download_dir, year, file_name)

    # Ensure year directory exists locally
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        if status:
            print(f"⬇️ Download {int(status.progress() * 100)}%.")

    print(f"✅ Saved to {local_path}")
    return local_path

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 9.2 MB/s eta 0:00:00


In [ ]:
#a code that downloads the files to a directory called content/reports ####

import os
import io
from googleapiclient.http import MediaIoBaseDownload

def download_all_chapters(download_dir="/content/reports", years=range(2001, 2016), chapters=range(1, 16)):
    """
    Downloads all chapters for all years to local environment.

    Returns:
        dict: {year: {chapter_number: local_path}}
    """
    all_paths = {}

    for year in years:
        year_str = str(year)
        all_paths[year_str] = {}

        for chapter in chapters:
            chapter_str = f"{chapter:02}"

            # Find matching file in df_files
            # Find matching .docx file in df_files
            match = df_files[
                (df_files["file_name"].str.endswith(".docx")) &   # only docx
                (df_files["file_name"].str.contains(chapter_str)) &
                (df_files["file_path"].str.contains(f"{year_str}/"))
            ]

            if match.empty:
                print(f"⚠️ Chapter {chapter_str} not found for year {year_str}")
                continue

            file_id = match.iloc[0]["file_id"]
            file_name = match.iloc[0]["file_name"]
            local_path = os.path.join(download_dir, year_str, f"{chapter_str}_{file_name}")

            # Ensure folder exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            # Download file
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(local_path, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"⬇️ Download {int(status.progress() * 100)}% for {file_name}")

            print(f"✅ Saved {file_name} to {local_path}")
            all_paths[year_str][chapter] = local_path

    return all_paths
download_all_chapters()

⬇️ Download 100% for 01.docx
✅ Saved 01.docx to /content/reports/2001/01_01.docx
⬇️ Download 100% for 02.docx
✅ Saved 02.docx to /content/reports/2001/02_02.docx
⬇️ Download 100% for 03.docx
✅ Saved 03.docx to /content/reports/2001/03_03.docx
⬇️ Download 100% for 04.docx
✅ Saved 04.docx to /content/reports/2001/04_04.docx
⬇️ Download 100% for 05.docx
✅ Saved 05.docx to /content/reports/2001/05_05.docx
⬇️ Download 100% for 06.docx
✅ Saved 06.docx to /content/reports/2001/06_06.docx
⬇️ Download 100% for 07.docx
✅ Saved 07.docx to /content/reports/2001/07_07.docx
⬇️ Download 100% for 08.docx
✅ Saved 08.docx to /content/reports/2001/08_08.docx
⬇️ Download 100% for 09.docx
✅ Saved 09.docx to /content/reports/2001/09_09.docx
⬇️ Download 100% for 10.docx
✅ Saved 10.docx to /content/reports/2001/10_10.docx
⬇️ Download 100% for 11.docx
✅ Saved 11.docx to /content/reports/2001/11_11.docx
⬇️ Download 100% for 12.docx
✅ Saved 12.docx to /content/reports/2001/12_12.docx
⬇️ Download 100% for 13.docx

{'2001': {1: '/content/reports/2001/01_01.docx',
  2: '/content/reports/2001/02_02.docx',
  3: '/content/reports/2001/03_03.docx',
  4: '/content/reports/2001/04_04.docx',
  5: '/content/reports/2001/05_05.docx',
  6: '/content/reports/2001/06_06.docx',
  7: '/content/reports/2001/07_07.docx',
  8: '/content/reports/2001/08_08.docx',
  9: '/content/reports/2001/09_09.docx',
  10: '/content/reports/2001/10_10.docx',
  11: '/content/reports/2001/11_11.docx',
  12: '/content/reports/2001/12_12.docx',
  13: '/content/reports/2001/13_13.docx',
  14: '/content/reports/2001/14_14.docx',
  15: '/content/reports/2001/15_15.docx'},
 '2002': {1: '/content/reports/2002/01_01.docx',
  2: '/content/reports/2002/02_02.docx',
  3: '/content/reports/2002/03_03.docx',
  4: '/content/reports/2002/04_04.docx',
  5: '/content/reports/2002/05_05.docx',
  6: '/content/reports/2002/06_06.docx',
  7: '/content/reports/2002/07_07.docx',
  8: '/content/reports/2002/08_08.docx',
  9: '/content/reports/2002/09_09.

In [ ]:
## code extracing 2001-2016 using libra - updated ###

ֵֵ!pip install python-docx
%apt-get install -y libreoffice
import os
import zipfile
import subprocess
from docx import Document
from PIL import Image
import json

graph_summary = {}

def trim_whitespace(image_path, save_path):
    """Trim extra whitespace around the image."""
    with Image.open(image_path) as im:
        im = im.convert("RGBA")
        bbox = im.getbbox()
        if bbox:
            im = im.crop(bbox)
        im.save(save_path)

def save_image_libre(img_data, img_ext, output_dir, idx, chapter, year):
    """Save image data; convert WMF/EMF to PNG using LibreOffice, else save directly."""
    tmp_filename = f"tmp_{idx}{img_ext}"
    tmp_path = os.path.join(output_dir, tmp_filename)

    with open(tmp_path, "wb") as f:
        f.write(img_data)

    serial_name = f"{idx}_{chapter}_{year}.png"
    save_path = os.path.join(output_dir, serial_name)

    if img_ext in [".wmf", ".emf"]:
        # Convert WMF/EMF → PNG using LibreOffice
        subprocess.run([
            "soffice", "--headless", "--convert-to", "png", "--outdir", output_dir, tmp_path
        ], check=True)

        exported_png = os.path.splitext(tmp_path)[0] + ".png"
        # Trim whitespace
        trim_whitespace(exported_png, save_path)
        os.remove(exported_png)
        os.remove(tmp_path)
    else:
      # Raster formats: save as PNG
      with Image.open(tmp_path) as im:
          # Convert CMYK to RGB if needed
          if im.mode == "CMYK":
              im = im.convert("RGB")
          im.save(save_path, "PNG")
      os.remove(tmp_path)


    return save_path

def extract_images_from_docx_tables(docx_path, output_dir, chapter, year, base_name="image"):
    os.makedirs(output_dir, exist_ok=True)
    doc = Document(docx_path)
    count = 0

    for table in doc.tables:
        first_row_text = " | ".join(cell.text.strip() for cell in table.rows[0].cells)
        if "תרשים" not in first_row_text:
            continue
        found_image = False
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    text_before = para.text.strip()
                    for run in para.runs:
                        for blip in run._element.xpath(".//a:blip"):
                            rId = blip.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed")
                            img_part = doc.part.related_parts[rId]
                            img_data = img_part.blob
                            img_ext = os.path.splitext(img_part.partname)[1].lower()
                            count += 1
                            img_filename = save_image_libre(img_data, img_ext, output_dir, count, chapter, year)
                            graph_summary[f'{count}_{chapter}_{year}'] = first_row_text
                            found_image = True
                            break
                        if found_image: break
                    if found_image: break
                if found_image: break
            if found_image: break

base_dir = "/content/reports"
save_dir = "/content/images"

for year in range(2016, 2017):
    year_folder = os.path.join(base_dir, str(year))
    for chapter in range(2,3):
        chapter_str = f"{chapter:02}"
        docx_file = os.path.join(year_folder, f"{chapter_str}_{chapter_str}.docx")
        if not os.path.exists(docx_file):
            continue
        out_path = os.path.join(save_dir, str(year), chapter_str)
        os.makedirs(out_path, exist_ok=True)
        base_name = f"{year}_{chapter_str}_"

        extract_images_from_docx_tables(docx_file, out_path, chapter, year, base_name=base_name)
        print(f"Finish extracting for year: {year}, chapter: {chapter}")

# Save JSON mapping
with open(os.path.join(save_dir, "graphs_summary.json"), "w", encoding="utf-8") as f:
    json.dump(graph_summary, f, ensure_ascii=False, indent=2)

print(f"Done! Extracted images are saved in '{save_dir}'")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libreoffice is already the newest version (1:7.3.7-0ubuntu0.22.04.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Finish extracting for year: 2016, chapter: 2
Done! Extracted images are saved in '/content/images'


In [25]:
## size sainty check ###
#3
from pathlib import Path
from PIL import Image

min_size = 200  # minimum acceptable width or height
problem_images = []

for img_path in Path(save_dir).rglob("*.png"):
    with Image.open(img_path) as im:

        w, h = im.size
        if w < min_size or h < min_size:
            problem_images.append((img_path, w, h))

print(f"Checked {len(list(Path(save_dir).rglob('*.png')))} images.")
if problem_images:
    print(f"⚠️ {len(problem_images)} images are smaller than {min_size}px:")
    for p, w, h in problem_images:
        print(f"  {p} → {w}x{h}")
else:
    print("✅ All images are larger than the threshold and should be readable.")



Checked 2119 images.
✅ All images are larger than the threshold and should be readable.


In [ ]:
from google.colab import drive
import shutil
import os

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Create a zip of your folder
zip_name = "graphs_2006-2008"  # will create graphs_2003-2005.zip
shutil.make_archive(zip_name, 'zip', "/content/graphs_to_csvs_2001-2016")

# 3️⃣ Move the zip into Google Drive
destination = "/content/drive/My Drive/graphs_2006-2008-csv.zip"
shutil.move(zip_name + ".zip", destination)

print(f"✅ File saved to Google Drive: {destination}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/graphs_to_csv_2001-2002.zip.zip'

In [ ]:
## extracting xlsx files correctly 2019, 2021-2024 by chapters ###

import os
import zipfile

def extract_embedded_excels(docx_path, output_dir="/content/extracted_files"):
    """
    Extract embedded Excel files from a .docx into a local directory.

    Args:
        docx_path (str): Path to the .docx file
        output_dir (str): Directory where extracted .xlsx files will be saved
    """
    os.makedirs(output_dir, exist_ok=True)

    with zipfile.ZipFile(docx_path, "r") as docx:
        # All embedded objects are inside word/embeddings/
        for file_name in docx.namelist():
            if file_name.startswith("word/embeddings/") and file_name.endswith(".xlsx"):
                # Create a unique name using the docx file's base name
                base_name = os.path.splitext(os.path.basename(docx_path))[0]
                out_file = os.path.join(output_dir, f"{base_name}_{os.path.basename(file_name)}")

                with docx.open(file_name) as source, open(out_file, "wb") as target:
                    target.write(source.read())
                print(f"Extracted: {out_file}")


extract_embedded_excels("/content/reports/2023/01_01.docx")

Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet.xlsx
Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet1.xlsx
Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet2.xlsx
Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet3.xlsx
Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet4.xlsx


In [ ]:
## extracting the names of the tables that it is in and naming the files correctly
# also extractingt the xlsx files into a diffrent counting pronting if it matched the counting ###
# for years 2019, 2021-2024

import os
import shutil
import zipfile
import pandas as pd
import json
from docx import Document

def extract_docx_tables_and_excels(docx_path, year, chapter, output_base="/content/extracted_files"):
    """
    Extract embedded Excel files from a DOCX, convert to CSV, and map table headers to them.

    Args:
        docx_path (str): Path to the .docx file
        year (str|int): Year to tag output files
        chapter (str|int): Chapter number to tag output files
        output_base (str): Base output directory
    """
    year, chapter = str(year), str(chapter)
    output_dir = os.path.join(output_base, year, chapter)

    # Reset output dir
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # ---------- Pass 1: Collect names from tables ----------
    # doc = Document(docx_path)
    # table_headers = []

    # for table in doc.tables:
    #     # Skip tables with no rows or no first cell text
    #     if len(table.rows) < 2:
    #         continue

    #     # Get the header text from the first row (row 0) if needed
    #     header_text = table.rows[0].cells[0].text.strip()
    #     if "תרשים" not in header_text:
    #         continue

    #     # Check only the second row (index 1)
    #     second_row = table.rows[1]
    #     has_content = False
    #     for cell in second_row.cells:
    #         # Check if the cell has any content (text, image, object)
    #         if len(cell._element) > 0 or cell.text.strip():
    #             print(len(cell._element))
    #             has_content = True
    #             print("has content: ",header_text)
    #             break

    # if not has_content:
    #     continue  # skip tables with an empty second row
    #     print("does not has content but do has 2+ cols: ", header_text)

    # # If second row has content, add the header
    # table_headers.append(header_text)
    doc = Document(docx_path)
    table_headers = []
    for table in doc.tables:
        if table.rows and table.cell(0,0).text:  # header exists
            if len(table.rows)==1:
              continue
            header_text = table.rows[0].cells[0].text.strip()
            if "תרשים" in header_text:
                # Repeat header as many times as there are columns
                #for _ in range(len(table.columns)):
                table_headers.append(header_text)

    print("Relevant table headers:", table_headers)

    # ---------- Pass 2: Extract embedded Excels and save as CSV ----------
    serial = 0
    csv_files = []
    with zipfile.ZipFile(docx_path, "r") as docx:
        for file_name in docx.namelist():
            if file_name.startswith("word/embeddings/") and file_name.endswith(".xlsx"):
                serial += 1
                out_csv = os.path.join(output_dir, f"{serial}_{chapter}_{year}.csv")

                # Save Excel temp
                excel_temp = os.path.join(output_dir, f"temp_{serial}.xlsx")
                with docx.open(file_name) as source, open(excel_temp, "wb") as target:
                    target.write(source.read())

                # Convert Excel -> CSV
                try:
                    xls = pd.ExcelFile(excel_temp)
                    # if multiple sheets -> concatenate into one CSV
                    df_all = []
                    for sheet in xls.sheet_names:
                        df = pd.read_excel(excel_temp, sheet_name=sheet)
                        df_all.append(df)
                    df_full = pd.concat(df_all, ignore_index=True)
                    df_full.to_csv(out_csv, index=False)
                    csv_files.append(out_csv)
                    print(f"Extracted: {out_csv}")
                except Exception as e:
                    print(f"⚠️ Could not convert {excel_temp}: {e}")
                finally:
                    os.remove(excel_temp)

    # ---------- Pass 3: Build JSON mapping ----------
    mapping = {}
    for idx, (csv_path, header) in enumerate(zip(csv_files, table_headers), start=1):
        key = f"{idx}_{chapter}_{year}"
        mapping[key] = header

    json_path = os.path.join(output_dir, f"{chapter}_{year}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(mapping, f, ensure_ascii=False, indent=2)

    # ---------- Sanity check ----------
    print("\n--- Sanity Check ---")
    print(f"Headers found: {len(table_headers)}")
    print(f"CSVs extracted: {len(csv_files)}")
    if len(table_headers) == len(csv_files):
        print("✅ Counts match.")
    else:
        print("⚠️ Mismatch between headers and CSVs!")

    print(f"\nJSON mapping saved at: {json_path}")
    return mapping



years= [2019,2021,2022,2023]
for year in years:
  for ch in range(1,15):
    extract_docx_tables_and_excels(f"/content/reports/{year}/{ch:02}_{ch:02}.docx",year,ch)



Relevant table headers: ["תרשים 1א': התפלגות הילדים בישראל, לפי דת (אחוזים) / 2023", "תרשים 1ב': יישובים* בהם ילדים מנו כ-25% ופחות או כ-50% ויותר מאוכלוסיית היישוב* / 2023", "תרשים 1ג': מספר הילדים (17-0) שקיבלו במהלך השנה מעמד של תושב ארעי או תושב קבע / 2020-2018", "תרשים 1ד': שיעור הילדים (17-0) הזרים שקיבלו במהלך השנה מעמד של תושב ארעי, לפי ארץ מוצא / 2020-2018", "תרשים 2א': שיעור הילדים העולים* מכלל אוכלוסיית העולים**, לפי תקופת עלייה / 2023-1948", 'תרשים 2ב\': מספר הילדים העולים (0-17) ילידי חו"ל לפי שנים לפי שנת עלייה / 2010 -\n2023', "תרשים 2ג': גיל הילדים (0-17) שעלו במהלך שנת 2023 (אחוזים)", 'תרשים 2ד\': שיעור הילדים העולים, ילידי חו"ל, לפי ארץ מוצא / 2023', "תרשים 2ה': מספר הילדים העולים* וילדי עולים** בגילאי 17-0 / 2008-2023***", "תרשים 2ו': התפלגות ילדי העולים* (17-0) וחלקם מכלל הילדים בישראל /\n2023-2010**", "תרשים 2ז': שיעור ילדי עולים*, לפי מספר הילדים במשפחה / 2023-2017", "תרשים 2ח': מספר ילדים* ממוצע במשפחות עולים עם ילדים בהשוואה לכלל המשפחות בישראל / 2023-2010**", "

{'1_1_2024': "תרשים 1א': התפלגות הילדים בישראל, לפי דת (אחוזים) / 2023",
 '2_1_2024': "תרשים 1ב': יישובים* בהם ילדים מנו כ-25% ופחות או כ-50% ויותר מאוכלוסיית היישוב* / 2023",
 '3_1_2024': "תרשים 1ג': מספר הילדים (17-0) שקיבלו במהלך השנה מעמד של תושב ארעי או תושב קבע / 2020-2018",
 '4_1_2024': "תרשים 1ד': שיעור הילדים (17-0) הזרים שקיבלו במהלך השנה מעמד של תושב ארעי, לפי ארץ מוצא / 2020-2018",
 '5_1_2024': "תרשים 2א': שיעור הילדים העולים* מכלל אוכלוסיית העולים**, לפי תקופת עלייה / 2023-1948",
 '6_1_2024': 'תרשים 2ב\': מספר הילדים העולים (0-17) ילידי חו"ל לפי שנים לפי שנת עלייה / 2010 -\n2023',
 '7_1_2024': "תרשים 2ג': גיל הילדים (0-17) שעלו במהלך שנת 2023 (אחוזים)",
 '8_1_2024': 'תרשים 2ד\': שיעור הילדים העולים, ילידי חו"ל, לפי ארץ מוצא / 2023',
 '9_1_2024': "תרשים 2ה': מספר הילדים העולים* וילדי עולים** בגילאי 17-0 / 2008-2023***",
 '10_1_2024': "תרשים 2ו': התפלגות ילדי העולים* (17-0) וחלקם מכלל הילדים בישראל /\n2023-2010**",
 '11_1_2024': "תרשים 2ז': שיעור ילדי עולים*, לפי מספר הילדים

In [ ]:
## filtering the relevant graphs from 2001-2016 ###

import os
import json
import shutil

# Paths
graphs_dir = "/content/graphs_2001_2016"
redundant_json_path = "/content/redundant_graphs.json"
graphs_summary_path = "/content/graphs_2001_2016/graphs_summary.json"
relevant_dir = "/content/relevant_graphs"
relevant_summary_path = "/content/relevant_graphs_summary.json"

# Load JSONs
with open(redundant_json_path, "r", encoding="utf-8") as f:
    redundant_map = json.load(f)

with open(graphs_summary_path, "r", encoding="utf-8") as f:
    graphs_summary = json.load(f)

# Ensure relevant directory exists
os.makedirs(relevant_dir, exist_ok=True)

# ---------- Filter relevant graphs ----------
relevant_summary = {}

for year in os.listdir(graphs_dir):
    year_path = os.path.join(graphs_dir, year)
    if not os.path.isdir(year_path):
        continue

    for chapter in os.listdir(year_path):
        chapter_path = os.path.join(year_path, chapter)
        if not os.path.isdir(chapter_path):
            continue

        for fname in os.listdir(chapter_path):
            if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".pdf")):
                continue

            key = os.path.splitext(fname)[0]  # remove extension, keep exact key

            if redundant_map.get(key, "NO") == "NO":  # only keep relevant
                # Copy to relevant directory
                new_year_dir = os.path.join(relevant_dir, year)
                new_chap_dir = os.path.join(new_year_dir, chapter)
                os.makedirs(new_chap_dir, exist_ok=True)
                shutil.copy2(os.path.join(chapter_path, fname), os.path.join(new_chap_dir, fname))

                # Add to relevant summary if exists
                if key in graphs_summary:
                    relevant_summary[key] = graphs_summary[key]

# Save relevant summary JSON
with open(relevant_summary_path, "w", encoding="utf-8") as f:
    json.dump(relevant_summary, f, ensure_ascii=False, indent=2)

print("✅ Filtering complete")
print(f"Relevant graphs saved under: {relevant_dir}")
print(f"Relevant summary saved as: {relevant_summary_path}")


✅ Filtering complete
Relevant graphs saved under: /content/relevant_graphs
Relevant summary saved as: /content/relevant_graphs_summary.json


In [ ]:
## extracting photos to csv's ###
## converting png to csv
# for years 2001-2016

!pip install anthropic
import os
import json
from pathlib import Path
from PIL import Image
import anthropic
import csv
import io
import re

# ---------- Config ----------
MAX_DIM = 1200   # resize larger images down
MIN_PIXELS = 50000  # skip if too small (< e.g., 200x250)
OUTPUT_DIR = "tables_out"

# Claude client
client = anthropic.Anthropic(
        api_key="" # replace with your actual API key
    )

# ---------- Utils ----------
def check_and_resize(path, max_dim=MAX_DIM, min_pixels=MIN_PIXELS):
    """
    Check image size and resize if too large.
    Skip if image too small.
    Returns PIL Image object or None.
    """
    img = Image.open(path)
    w, h = img.size
    if w * h < min_pixels:
        return None  # too small → skip
    scale = max_dim / max(w, h)
    if scale < 1:  # downscale
        new_size = (int(w * scale), int(h * scale))
        img = img.resize(new_size, Image.LANCZOS)
    print("proccesed img")
    return img

import base64
from io import BytesIO

def pil_to_base64(img):
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

def send_to_claude(image, prompt):
    """
    Send single image and prompt to Claude Sonnet 2.7 and return text output.
    """
    img_b64 = pil_to_base64(image)



    message = client.messages.create(
    # model="claude-sonnet-4-20250514",
    model="claude-3-7-sonnet-20250219",
    # model= "claude-3-haiku-20240307",
    max_tokens=4096,
    temperature=1,
    system="data extraction ",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": img_b64
                    }
                },
                {
                    "type": "text",
                    "text": prompt
                }
                ]
            }
        ]
    )
    print(message.content)
    print("after Api call")
    return message.content[0].text.strip()



# ---------- Main pipeline ----------
def process_images(img_dir, graphs_json_path, output_dir, chapter_num=None, year_num=None):
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Load graph names JSON
    with open(graphs_json_path, "r", encoding="utf-8") as f:
        graphs_map = json.load(f)

    if not os.path.exists(img_dir):
        print(f"⚠️ Skipping missing directory: {img_dir}")
        return

    # Collect valid images
    valid_images = []
    for fname in sorted(os.listdir(img_dir)):

        if not fname.lower().endswith((".png", ".jpg", ".jpeg")):
            continue
        path = os.path.join(img_dir, fname)
        img = check_and_resize(path)
        if img is None:
            print(f"⚠️ Skipping too-small image: {fname}")
            continue
        img = Image.open(path)
        if img is None:
            print(f"Skipping could not open: {fname}")
            continue
        valid_images.append((fname, img))

    if not valid_images:
        print("No valid images found in the directory.")
        return

    # Process each image individually
    for fname, img in valid_images:
        key = os.path.splitext(fname)[0]  # -> "3_4_2004"
        graph_name = graphs_map.get(key)
        print(graph_name)
        print(key)
        prompt =(
                   "You are a data analyst expert.\nTask: Extract a table from the attached image into a CSV file.\n\nRules:\n-All images contain Hebrew text.\n- if there is a number seperate with commas (e.g. 110,000) change it to non comma (110000).\n-Do not translate or change any values. Preserve all Hebrew exactly as in the table.\n-If a cell cannot be parsed, leave it empty (do not invent values).\n- if you cant read the hebrew do not invent words. use the graph name for context.\n-Some tables have section headers (e.g. סך הכל, חינוך עברי, חינוך ערבי).\nIn those cases, add a new column at the beginning called קבוצה, and repeat the section name for each row under it.\n- section headers cant be a header- it either a section header or a first row header.\n-If there is an informative sentence below/under the graph, put it as a whole row in the end of the CSV,leave an empty row before.\n-The header of the table for context is: {graph_name}\n-Output format: strict CSV (comma-separated, UTF-8).\n-All rows in the CSV must have exactly the same number of columns as the header row. If there are totals, footnotes, or sentences under the graph, fit them into the same number of columns (by using empty cells).\n-Do not wrap in Markdown or code fences.\n-If you cannot parse the full table, return exactly: null.\n\nReturn only one of:\nA valid CSV table, or\nThe string: null\n!! Do not return any text that is not a CSV file or null !!"
                  )

        print(f"Processing image: {fname} (graph: {graph_name})")
        result = send_to_claude(img, prompt).strip()
        print(result)
        # Skip saving if result is null
        if result.lower() == "null":
            print(f"⚠️ Skipping {fname} — Claude returned null (unreadable table).")
            continue

        try:
            # while re.search(r'(\d+),(\d{3})(?!\d)', result):
            #   result = re.sub(r'(\d+),(\d{3})(?!\d)', r'\1\2', result)
            reader = csv.reader(io.StringIO(result))
            rows = list(reader)
            if not rows or all(len(row) <=1 for row in rows):
                print(f"⚠️ Skipping {fname} — result is not a valid CSV.")
                continue

            rows[0] = [re.sub(r'\*+$', '', col).strip() for col in rows[0]]
            max_cols = max(len(row) for row in rows)
            for i, row in enumerate(rows):
                if len(row) < max_cols:
                    rows[i] = row + [""] * (max_cols - len(row))
                    print(f"⚠️ Row {i} in {fname} padded with empty strings.")
            outname = fname.rsplit(".", 1)[0] + ".csv"
            outpath = os.path.join(output_dir, outname)
            with open(outpath, "w", encoding="utf-8", newline="") as f:
                 writer = csv.writer(f)
                 writer.writerows(rows)

            print(f"✅ Saved CSV: {outname}")
        except Exception as e:
            print(f"⚠️ Skipping {fname} — CSV parse error: {e}")
            continue



for year in range(2001,2016):
    print(f"Processing year: {year}")
    for chap in range(1,16):
        print(f"Processing chapter: {chap}")
        process_images(f"/content/relevant_graphs/{year}/{chap:02d}",
                      "/content/relevant_graphs_summary.json",
                      f"/content/graphs_to_csvs_2001-2016/{year}/{chap:02d}",chap,year)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 26.1 MB/s eta 0:00:00
Processing year: 2009
Processing chapter: 1
proccesed img
proccesed img
יישובים שבהם  הילדים מונים עד 25% או 55% ויותר 
מהאוכלוסייה
דצמבר 2008 | תרשים 1ב'
2_1_2009
Processing image: 2_1_2009.png (graph: יישובים שבהם  הילדים מונים עד 25% או 55% ויותר 
מהאוכלוסייה
דצמבר 2008 | תרשים 1ב')
[TextBlock(citations=None, text='ישוב,סוג\nכרמיאל - ק. מוצקין,●\nק. ביאליק,●\nנצרת עילית,●\nנהריה,●\nחיפה,●\nק. ים,●\nהרצליה - גבעתיים,●\nתל-אביב-יפו,●\nרמת גן,●\nבת ים,●\nחולון,●\nראשל"צ,●\nנתניה,●\nמודיעין - עילית,■\nבית"ר עילית,■\nרהט,■\nכסייפה,■\nתל שבע,■\nערערה - בנגב,■\nשגב שלום,■\nחורה,■\nאלעד,■\nביטחוני,33.0%', type='text')]
after Api call
ישוב,סוג
כרמיאל - ק. מוצקין,●
ק. ביאליק,●
נצרת עילית,●
נהריה,●
חיפה,●
ק. ים,●
הרצליה - גבעתיים,●
תל-אביב-יפו,●
רמת גן,●
בת ים,●
חולון,●
ראשל"צ,●
נתניה,●
מודיעין - עילית,■
בית"ר עילית,■
רהט,■
כסייפה,■
תל שבע,■
ערערה - בנגב,■
שגב שלום,■
חורה,■
אלעד,■
ביטחוני,33.0%
✅ Saved CSV: 2_1_20

In [ ]:
##checking how many graphs hasnt been extracted ###

import os
import csv

base_dir = "/content/graphs_to_csvs_2001-2016"

for year in range(2003, 2006):
    for chap in range(1, 16):
        dir_path = os.path.join(base_dir, str(year), f"{chap:02d}")
        if not os.path.exists(dir_path):
            continue

        for fname in os.listdir(dir_path):
            if not fname.endswith(".csv"):
                continue
            file_path = os.path.join(dir_path, fname)

            # Read CSV
            with open(file_path, "r", encoding="utf-8") as f:
                reader = csv.reader(f)
                rows = list(reader)

            # Check if any cell contains "null" (case-insensitive)
            contains_null = any("null" in (cell or "").lower() for row in rows for cell in row)

            if contains_null:
                # Overwrite CSV with placeholder
                # with open(file_path, "w", encoding="utf-8", newline="") as f:
                    # writer = csv.writer(f)
                    # writer.writerow([f"⚠️ Graph hasn’t been extracted: {fname}"])
                print(f"⚠️ Replaced CSV with message: {file_path}")
